In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import random
from sklearn.neighbors import KDTree
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn import metrics
import multiprocessing as mp

In [2]:
sp_10path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/results/sp/sp_lp_10.pkl'
with open(sp_10path, 'rb') as file:
    sp10 = pickle.load(file) 

In [20]:
index = []
for i,value in enumerate(sp10[1]):
    if value != float('inf'):
        index.append(i)
    else:
        pass

In [24]:
len(index),len(sp10[1])

(431762, 443280)

In [22]:
predict_label = []
true_label = []
for i in index:
    predict_label.append(sp10[1][i])
    true_label.append(sp10[0][i])

In [23]:

precision, recall, thresholds = metrics.precision_recall_curve(true_label,predict_label)
roc,pr= metrics.roc_auc_score(true_label,predict_label),metrics.auc(recall,precision)
print(roc,pr)


0.8044485176045822 0.8170182040940206


In [ ]:
with open(record_path, 'wb') as file:
    pickle.dump([roc, pr], file)
# Close the file
file.close()

In [4]:
root = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/datasets/ppi/'
record_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/results/sp/sp_bi_cad.pkl'
temp_neighbor_path = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/results/sp/sp_bi_cad.csv'
with open('/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/test/cad_node.pkl', 'rb') as file:
    group_node = pickle.load(file) 

sparse_i = np.loadtxt(root+'sparse_i.txt')
sparse_j = np.loadtxt(root+'sparse_j.txt')
sparse_w = np.loadtxt(root+'sparse_w.txt')
 
sparse_i = sparse_i.astype(int)
sparse_j = sparse_j.astype(int)
sparse_w = sparse_w.astype(float)
 
string_score_transform = np.vectorize(lambda x: -np.log(x/1000))
transformed_w = string_score_transform(sparse_w)
edges_and_weights = zip(sparse_i, sparse_j,transformed_w)
G = nx.Graph()
# Use from_edgelist to directly create the graph from edges
G.add_weighted_edges_from(edges_and_weights)

In [7]:
if 17729 in list(sparse_j):
    print('1')

1


In [8]:
if G.has_node(17729):
    print("1")

1


In [13]:
k_max = 100
shortest_paths_weighted = nx.shortest_path_length(G, source=17729)


neighborlist = list(shortest_paths_weighted.keys())
neighborlist.pop(0)
neighbor=neighborlist[:k_max]

distancelist = list(shortest_paths_weighted.values())
distancelist.pop(0)
dist=distancelist[:k_max]

In [3]:
ks = [3,5]
group_node = group_node[:100]

In [8]:
group_node[-1]

17312

In [5]:
def get_neighbor_list(args):
    start_point, G, k_max = args
 
    shortest_paths_weighted = nx.shortest_path_length(G, source=start_point)
    start=k_max * [start_point]
 
    neighborlist = list(shortest_paths_weighted.keys())
    neighborlist.pop(0)
    neighbor=neighborlist[:k_max]
 
    distancelist = list(shortest_paths_weighted.values())
    distancelist.pop(0)
    dist=distancelist[:k_max]
    return start, neighbor, dist
 
def parallel_worker(args):
    return get_neighbor_list(args)
 
if __name__ == "__main__":
    num_processes = mp.cpu_count()
    mp.freeze_support()
 
    pool = mp.Pool(processes=num_processes)
    args_list = [(node, G, max(ks)) for node in group_node]
   
    results = pool.map(parallel_worker, args_list)
    
    pool.close()
    pool.join()
 
    starts = []
    neighbors = []
    dists = []
 
    for result in results:
        start, neighbor, dist = result
        starts.extend(start)
        neighbors.extend(neighbor)
        dists.extend(dist)
    neighbor_df = pd.DataFrame({'start': start, 'neighbor': neighbor, 'distance': dist})

KeyboardInterrupt: 

In [7]:
results

NameError: name 'results' is not defined